Day 1 Output:

Determine which teams go into which pools
Write the results to the pool masters

Men's: 

Top 2 teams from each pool go into Gold (Total: 16)
Next 12 teams go into Silver (2 pools, 6 teams, 1 game 0-21)
Next 12 go into Bronze (2 pools, 6 teams, 1 game 0-21)

Women's:

Top 16 teams go into Gold (First, Top 2 from each team, but the last four teams will challenge match for the last two spots in Gold)
Next 16 teams go into Silver (4x4)
Next 12 teams go into Bronze playoffs


In [122]:
# function to finalize pools for 2nd Round
# input: list of team names (sorted by ranking), desired shape

import numpy as np
import pandas as pd

# Retrieve pool rankings
m_pools = ['M' + str(i) for i in range(1,9)]
f_pools = ['W' + str(i) for i in range(1,10)]

input_ws = ["NY Mini 2017 - 1st Round"]
output_ws = ["GOLD NY Mini 2017 - Day 2 Pools", "SILVER NY Mini 2017 - Day 2 Pools", "BRONZE NY Mini 2017 - Day 2 Pools"]

def snaked_teams(team_names, rows=4, cols=4):
    #check for correct size
    if len(team_names) != rows * cols:
        print ("Shape size mismatch. Cannot perform this function for " + str(len(team_names)) + " teams with a " 
               + str(rows) + " x " + str(cols) + " matrix")
        return None
    
    output = []
    
    for i in range(rows):
        row = team_names[:cols]
        if i % 2 == 1:
            row.reverse()
            
        output.append(row)
        
        team_names=team_names[cols:]

    return pd.DataFrame(output) # need to start index at 1 not zero
        

In [101]:
#returns list of challenge matches given list of team names already sorted
# assuming challenges are 1v1
# start at the ends and pairs them up
# can also be used for seeding playoffs

def challenge_matches(team_names):
    #16 teams (0,1,2... 15)
    output = []
    for i in range(int(len(team_names)/2)):
        output.append(team_names[i] + " vs " + team_names[-i-1])
    return output

In [78]:
# Select the last four of a sorted DataFrame
def last_four(df):
    n = len(df)
    return df.iloc[n-4:n]['Team']

Run code below to populate Day 2 Men's Gold, Silver, Bronze Brackets (mens_raw for overall)

In [168]:
# Data Frame Construction: (Need this info)
# Pool #, Placement, Team Name, Games (Ratio), Points (Ratio)

import pandas as pd
from ReadWriteGDocs import write_gdoc, read_gdoc_range, write_gdoc_cell

mens_raw = pd.DataFrame()
c=['Ranking', 'Team', 'Games', 'Points']

#Grab data from GDOC
for i in m_pools:
    df=pd.DataFrame(read_gdoc_range(input_ws[0], "Court " + i, 27, 2, 31,5), columns=c)
    df.insert(0, 'Pool', i)
    mens_raw=mens_raw.append(df)

# #Convert dtype so the sorting is accurate
    
mens_raw['Games']=pd.DataFrame(mens_raw['Games'], dtype=int)
mens_raw['Points']=pd.DataFrame(mens_raw['Points'], dtype=int)
mens_raw['Ranking']=pd.DataFrame(mens_raw['Ranking'], dtype=int)
mens_raw=mens_raw.sort_values(['Ranking', 'Games', 'Points'], ascending=[True, False, False])

#Form the Gold, Silver Bronze teams

m_gold = mens_raw[(mens_raw['Ranking'] == 2) | (mens_raw['Ranking'] == 1)]


# #Sort by Ranking, Games, Points
    
m_gold=m_gold.sort_values(['Ranking', 'Games', 'Points'], ascending=[True, False, False])
m_gold=m_gold.reset_index(drop=True)

mens_temp = mens_raw[ (mens_raw['Ranking'] != 1) & (mens_raw['Ranking'] != 2) ].sort_values(['Ranking', 'Games', 'Points'], ascending=[True, False, False])
mens_temp=mens_temp.reset_index(drop=True)

m_silver = mens_temp.iloc[range(12)]
m_bronze = mens_temp.iloc[range(12,len(mens_temp))]
m_bronze = m_bronze.reset_index(drop=True)

m_gold_snaked = snaked_teams(list(m_gold['Team']))
m_silver_snaked = snaked_teams(list(m_silver['Team']), 6, 2)
m_bronze_snaked = snaked_teams(list(m_bronze['Team']), 6, 2)

for index,row in m_gold_snaked.iterrows():
    for i in range(len(row)):
        write_gdoc_cell(row[i], output_ws[0], "Men's Gold", index+4, i+2)
        
print("Finished Men's Gold Bracket")
        
for index,row in m_silver_snaked.iterrows():
    for i in range(len(row)):
        write_gdoc_cell(row[i], output_ws[1], "Men's Silver", index+4, i+2)
        
print("Finished Men's Silver Bracket")
        
for index,row in m_bronze_snaked.iterrows():
    for i in range(len(row)):
        write_gdoc_cell(row[i], output_ws[1], "Men's Bronze", index+4, i+2)
        
print("Finished Men's Bronze Bracket")

In [183]:
# Data Frame Construction: (Need this info)
# Pool #, Placement, Team Name, Games (Ratio), Points (Ratio)

import pandas as pd
from ReadWriteGDocs import write_gdoc, read_gdoc_range, write_gdoc_cell

womens_raw = pd.DataFrame()
c=['Ranking', 'Team', 'Games', 'Points', 'Games Ratio', 'Points Ratio']

#Grab data from GDOC
for i in f_pools:
    if i == 'W9':
        df=pd.DataFrame(read_gdoc_range(input_ws[0], "Court " + i, 27, 2, 30, 7), columns=c)
    else:
        df=pd.DataFrame(read_gdoc_range(input_ws[0], "Court " + i, 27, 2, 31,7), columns=c)
    
    df.insert(0, 'Pool', i)
    
    womens_raw=womens_raw.append(df)

# #Convert dtype so the sorting is accurate
    
womens_raw['Games']=pd.DataFrame(womens_raw['Games'], dtype=int)
womens_raw['Points']=pd.DataFrame(womens_raw['Points'], dtype=int)
womens_raw['Ranking']=pd.DataFrame(womens_raw['Ranking'], dtype=int)
womens_raw=womens_raw.sort_values(['Ranking', 'Games Ratio', 'Points Ratio'], ascending=[True, False, False])
womens_raw=womens_raw.reset_index(drop=True)

#Form the Gold, Silver Bronze teams

w_gold = womens_raw[(womens_raw['Ranking'] == 2) | (womens_raw['Ranking'] == 1)]
last4 = last_four(w_gold)
print("Women's Gold Challenge Matches are: " + str(challenge_matches(list(last4))))

w_silver = womens_raw.iloc[range(len(w_gold),32)]
w_silver = w_silver.reset_index(drop=True)

w_bronze = womens_raw.iloc[range(32,len(womens_raw))]
w_bronze = w_bronze.reset_index(drop=True)


Women's Gold Challenge Matches are: ['U vs AE', 'AH vs Q']


In [187]:
#After the challenge matches:
#Drop the two teams from Gold
#Add the two teams to Silver
#Sort again

lost = ['U', 'AE']

w_silver = w_silver.append(w_gold.iloc[w_gold.index[(w_gold['Team'] == lost[0])]])
w_silver = w_silver.append(w_gold.iloc[w_gold.index[(w_gold['Team'] == lost[1])]])
w_silver = w_silver.sort_values(['Ranking', 'Games Ratio', 'Points Ratio'], ascending=[True, False, False])
w_silver = w_silver.reset_index(drop=True)

w_gold = w_gold.drop(w_gold.index[(w_gold['Team'] == lost[0]) | (w_gold['Team'] == lost[1])])
w_gold = w_gold.reset_index(drop=True)

#Determine the Pools after challenge matches

f_gold_snaked = snaked_teams(list(w_gold['Team']))
f_silver_snaked = snaked_teams(list(w_silver['Team']))
f_bronze_snaked = snaked_teams(list(w_bronze['Team']), 12, 1)

for index,row in f_gold_snaked.iterrows():
    for i in range(len(row)):
        write_gdoc_cell(row[i], output_ws[0], "Women's Gold", index+4, i+2)
        
print("Finished Women's Gold Bracket")
        
for index,row in f_silver_snaked.iterrows():
    for i in range(len(row)):
        write_gdoc_cell(row[i], output_ws[1], "Women's Silver", index+4, i+2)
        
print("Finished Women's Silver Bracket")

for index,row in f_bronze_snaked.iterrows():
    write_gdoc_cell(row[0], output_ws[2], "Women's Bronze", index+4, 2)
    
print("Finished Women's Bronze Bracket")

Finished Women's Gold Bracket
Finished Women's Silver Bracket
Finished Women's Bronze Bracket
